In [1]:
import os

In [2]:
%pwd

'c:\\Users\\drako\\Documents\\Data Projects\\Dogs-Cats-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\drako\\Documents\\Data Projects\\Dogs-Cats-Classification'

In [8]:
from cnnClassifier.pipeline.stage_02_data_preparation import ImageResizingStage, logger

# Specify input and output directories for cats and dogs
cats_input_dir = 'artifacts\data_ingestion\cats'
dogs_input_dir = 'artifacts\data_ingestion\dogs'
cats_output_dir = 'artifacts\data_ingestion\cats_resized'
dogs_output_dir = 'artifacts\data_ingestion\dogs_resized'

# Create an instance of the ImageResizingStage class
resizing_stage = ImageResizingStage([cats_input_dir, dogs_input_dir], [cats_output_dir, dogs_output_dir])

# Execute the image resizing operation
resizing_stage.run()

logger.info("Image Resizing Stage completed")


[2024-01-31 20:36:38,805: INFO: stage_02_data_preparation: Image Resizing Stage started]
Resized: cat_10.jpg
Resized: cat_100.jpg
Resized: cat_101.jpg
Resized: cat_102.jpg
Resized: cat_103.jpg
Resized: cat_104.jpg
Resized: cat_11.jpg
Resized: cat_111.jpg
Resized: cat_12.jpg
Resized: cat_120.jpg
Resized: cat_123.jpg
Resized: cat_129.jpg
Resized: cat_13.jpg
Resized: cat_132.jpg
Resized: cat_133.jpg
Resized: cat_134.jpg
Resized: cat_137.jpg
Resized: cat_138.jpg
Resized: cat_139.jpg
Resized: cat_14.jpg
Resized: cat_141.jpg
Resized: cat_146.jpg
Resized: cat_148.jpg
Resized: cat_149.jpg
Resized: cat_15.jpg
Resized: cat_151.jpg
Resized: cat_152.jpg
Resized: cat_153.jpg
Resized: cat_159.jpg
Resized: cat_16.jpg
Resized: cat_160.jpg
Resized: cat_161.jpg
Resized: cat_163.jpg
Resized: cat_164.jpg
Resized: cat_165.jpg
Resized: cat_168.jpg
Resized: cat_169.jpg
Resized: cat_170.jpg
Resized: cat_172.jpg
Resized: cat_177.jpg
Resized: cat_179.jpg
Resized: cat_180.jpg
Resized: cat_183.jpg
Resized: cat_18

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [15]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e

[2024-01-31 22:34:31,646: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-31 22:34:31,648: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-31 22:34:31,648: INFO: common: created directory at: artifacts]
[2024-01-31 22:34:31,649: INFO: common: created directory at: artifacts/prepare_base_model]
58889256/58889256 [==============================] - 1s 0us/step
[2024-01-31 22:34:33,347: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                 

c:\Users\drako\anaconda3\envs\dogs_cats\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
